### Import Libraries

The first step is to import the libraries that we will need.

In [1]:
import json
import os
import datetime

import wallaroo
from wallaroo.object import EntityNotFoundError

# used to display dataframe information without truncating
from IPython.display import display
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [2]:
wallaroo.__version__

'2023.2.0rc4'

### Initialize connection

Start a connect to the Wallaroo instance and save the connection into the variable `wl`.

In [3]:
# Login through local Wallaroo instance

wl = wallaroo.Client()

wallarooPrefix = "doc-test"
wallarooSuffix = "wallaroocommunity.ninja"

wl = wallaroo.Client(api_endpoint=f"https://{wallarooPrefix}.api.{wallarooSuffix}", 
                    auth_endpoint=f"https://{wallarooPrefix}.keycloak.{wallarooSuffix}", 
                    auth_type="sso")

### Set Configurations

The following will set the workspace, model name, and pipeline that will be used for this example.  If the workspace or pipeline already exist, then they will assigned for use in this example.  If they do not exist, they will be created based on the names listed below.

In [4]:
workspace_name = 'moto2'
pipeline_name = 'bikedaypipe'
model_name = 'bikedaymodel'

## Set the Workspace and Pipeline


In [5]:
def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

def get_pipeline(name):
    try:
        pipeline = wl.pipelines_by_name(name)[0]
    except EntityNotFoundError:
        pipeline = wl.build_pipeline(name)
    return pipeline

workspace = get_workspace(workspace_name)

wl.set_current_workspace(workspace)

pipeline = get_pipeline(pipeline_name)
# pipeline

### Upload Model

This model is a simple ARIMA(1,0,1) with no exogenous variables, hard-coded to forecast seven days out.

Note that this package is being specified as a `python` configuration.

In [6]:
model_file_name = 'forecast.py'

bike_day_model = wl.upload_model(model_name, model_file_name).configure(runtime="python")

### Deploy the Pipeline

We will now add the uploaded model as a step for the pipeline, then deploy it.

In [7]:
# Set the deployment to allow for additional engines to run
deploy_config = (wallaroo.DeploymentConfigBuilder()
                        .replica_count(1)
                        .replica_autoscale_min_max(minimum=2, maximum=5)
                        .cpus(1)
                        .memory("1Gi")
                        .build()
                    )

# pipeline = wl.build_pipeline('pipeline')
pipeline.add_model_step(bike_day_model).deploy(deployment_config = deploy_config)

name,bikedaypipe
created,2023-05-22 17:16:59.400429+00:00
last_updated,2023-05-22 17:38:45.873350+00:00
deployed,True
tags,
versions,"9565e1c4-92d8-4785-a7b8-580565edfa57, ed30b806-102f-43c0-8be1-2b4c4579f497, 6aede63f-cf29-4cb1-9cd4-2c5548a11357, a061b5a3-6ed3-4e90-b950-90d7bb9e57d1, 441faa7f-1957-4bf0-b4fe-eaa79a671fbe, 52e355a8-43a5-4a23-9aec-320e0d7b6f36, 05a5bad1-878b-43d1-b81b-59874111cad8"
steps,bikedaymodel


In [8]:
model_file_name = 'forecast.py'
bike_day_model_two = wl.upload_model(model_name, model_file_name).configure(runtime="python")

In [9]:
deployment_config = wallaroo.DeploymentConfigBuilder().replica_count(2).cpus(1).memory("1Gi").build()
pipeline_two = wl.build_pipeline('pipeline-two')
pipeline_two.add_model_step(bike_day_model).deploy(deployment_config = deployment_config)

name,pipeline-two
created,2023-05-22 17:38:58.020481+00:00
last_updated,2023-05-22 17:38:58.660875+00:00
deployed,True
tags,
versions,"912086bd-2b11-4ffa-9bd8-cba9b17736f8, 9dfb3c74-2979-4196-98d1-31e88a8e83c2"
steps,bikedaymodel


In [10]:
pipeline_two._deployment._url()

'https://doc-test.api.wallaroocommunity.ninja/v1/api/pipelines/infer/pipeline-two-18/pipeline-two'

In [11]:
pipeline._deployment._url()

'https://doc-test.api.wallaroocommunity.ninja/v1/api/pipelines/infer/bikedaypipe-12/bikedaypipe'

## run 
./streamer http://engine-lb.pipeline-two-257:29502/pipelines/pipeline-two 2 bikerental_moto_demo_python/testdata_dict.json

### Run Inference
Run a test inference to verify the pipeline is operational.

In [12]:
results = pipeline.infer_from_file('testdata_dict.json', data_format="custom-json")
display(results)
# results[0].data()

[{'forecast': [1764, 1749, 1743, 1741, 1740, 1740, 1740]}]

### Undeploy the Pipeline

Undeploy the pipeline and return the resources back to the Wallaroo instance.

In [13]:
pipeline.undeploy()

name,bikedaypipe
created,2023-05-22 17:16:59.400429+00:00
last_updated,2023-05-22 17:38:45.873350+00:00
deployed,False
tags,
versions,"9565e1c4-92d8-4785-a7b8-580565edfa57, ed30b806-102f-43c0-8be1-2b4c4579f497, 6aede63f-cf29-4cb1-9cd4-2c5548a11357, a061b5a3-6ed3-4e90-b950-90d7bb9e57d1, 441faa7f-1957-4bf0-b4fe-eaa79a671fbe, 52e355a8-43a5-4a23-9aec-320e0d7b6f36, 05a5bad1-878b-43d1-b81b-59874111cad8"
steps,bikedaymodel
